In [8]:
import csv 
filename = 'Electric_Vehicle_Charging_Stations.csv'
keys = {'Station Name', 'New Georeferenced Column'}
records =[]
with open(filename, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        records.append({key: row[key] for key in keys})

In [9]:
print(records[0])

{'Station Name': 'BMW OF DARIEN', 'New Georeferenced Column': 'POINT (-73.4764687 41.072882)'}


In [10]:
for record in records :
    longitude, latitude = record['New Georeferenced Column'].split("(")[-1].split(")")[0].split()
    record['longitude'] = float(longitude)
    record['latitude'] = float(latitude)


In [11]:
records[0]

{'Station Name': 'BMW OF DARIEN',
 'New Georeferenced Column': 'POINT (-73.4764687 41.072882)',
 'longitude': -73.4764687,
 'latitude': 41.072882}

MAPS USING FOLIUM

In [1]:
!pip install folium

  Obtaining dependency information for folium from https://files.pythonhosted.org/packages/ae/6d/18a7546e1748ecdd6ed7cd00d3f183faf1df08bd4f5e5e0eb3e72458b862/folium-0.17.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for branca>=0.6.0 from https://files.pythonhosted.org/packages/75/ca/6074ab4a04dd1a503201c18091b3426f3709670115fae316907a97f98d75/branca-0.7.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.4/108.4 kB 5.4 MB/s eta 0:00:00


In [19]:
import folium 
from folium.plugins import FastMarkerCluster
map = folium.Map(location =[41.5025,-72.69997], zoom_start=9)
latitudes = [a['latitude'] for a in records]
longitudes = [a['longitude']for a in records]
for record in records:
    coords =(record['latitude'], record['longitude'])
    folium.Marker(coords, popup=record['Station Name']).add_to(map)

FastMarkerCluster(data=list(zip(latitudes, longitudes))).add_to(map)
map

In [14]:
map

In [1]:
import folium 
from folium.plugins import FastMarkerCluster
from sklearn.cluster import KMeans
import numpy as np

# Assuming 'records' is defined and contains latitude and longitude
latitudes = np.array([a['latitude'] for a in records])
longitudes = np.array([a['longitude'] for a in records])
coordinates = np.column_stack((latitudes, longitudes))

# K-Means clustering
num_clusters = 5  # You can adjust the number of clusters
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(coordinates)
clusters = kmeans.labels_

# Create map
map = folium.Map(location=[41.5025, -72.69997], zoom_start=9)

# Add clustered markers
for i in range(num_clusters):
    cluster_coords = coordinates[clusters == i]
    if len(cluster_coords) > 0:
        folium.Marker(
            location=cluster_coords.mean(axis=0),
            popup=f'Cluster {i+1}: {len(cluster_coords)} stations'
        ).add_to(map)

# Optionally, use FastMarkerCluster for individual markers
FastMarkerCluster(data=list(zip(latitudes, longitudes))).add_to(map)

map


NameError: name 'records' is not defined

In [3]:
import csv

filename = 'Electric_Vehicle_Charging_Stations.csv'
# Use a list to maintain order
keys = ['Station Name', 'New Georeferenced Column']
records = []

with open(filename, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        # Split the 'New Georeferenced Column' into latitude and longitude
        if row['New Georeferenced Column']:
            lat_long = row['New Georeferenced Column'].split(',')
            latitude = float(lat_long[0].strip())
            longitude = float(lat_long[1].strip())
            records.append({
                'Station Name': row['Station Name'],
                'latitude': latitude,
                'longitude': longitude
            })

# Now 'records' contains the relevant data


ValueError: could not convert string to float: 'POINT (-73.4764687 41.072882)'

fast marker cluster

In [1]:
import folium
from math import sqrt

# Function to calculate Euclidean distance between two points
def distance(point1, point2):
    return sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# Function to perform basic clustering based on proximity (brute-force method)
def cluster_coordinates(coords, threshold=0.05):
    clusters = []
    
    for coord in coords:
        added_to_cluster = False
        for cluster in clusters:
            if distance(coord, cluster[0]) < threshold:
                cluster.append(coord)
                added_to_cluster = True
                break
        if not added_to_cluster:
            clusters.append([coord])
    
    return clusters

# Sample records with latitude and longitude
records = [
    {'Station Name': 'Station 1', 'latitude': 41.5025, 'longitude': -72.69997},
    {'Station Name': 'Station 2', 'latitude': 41.7025, 'longitude': -72.89997},
    {'Station Name': 'Station 3', 'latitude': 41.6025, 'longitude': -72.79997},
    {'Station Name': 'Station 4', 'latitude': 41.5028, 'longitude': -72.69987},
    {'Station Name': 'Station 5', 'latitude': 41.5030, 'longitude': -72.69977},
]

# Extract coordinates from records
coords = [(record['latitude'], record['longitude']) for record in records]

# Cluster the coordinates using a distance threshold
clusters = cluster_coordinates(coords)

# Initialize map
map = folium.Map(location=[41.5025, -72.69997], zoom_start=9)

# Add clustered markers
for cluster in clusters:
    # For each cluster, place a marker at the mean position of the cluster
    latitudes = [c[0] for c in cluster]
    longitudes = [c[1] for c in cluster]
    avg_lat = sum(latitudes) / len(latitudes)
    avg_lng = sum(longitudes) / len(longitudes)
    
    # Create a marker for the cluster
    folium.Marker(location=[avg_lat, avg_lng], popup=f"Cluster of {len(cluster)} stations").add_to(map)

# Display the map
map


In [6]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/find-nearest', methods=['GET'])
def find_nearest():
    lat = float(request.args.get('lat'))
    lon = float(request.args.get('lon'))
    
    # Find nearest station using the find_nearest_station function
    nearest_station, distance = find_nearest_station(lat, lon, records)
    
    # Return the result as JSON
    return jsonify({
        'station_name': nearest_station['Station Name'],
        'distance': distance
    })

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
Traceback (most recent call last):
  File "/Users/shashikumarezhil/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 15, in <module>
    from ipykernel import kernelapp as app
  File "/Users/shashikumarezhil/anaconda3/lib/python3.11/site-packages/ipykernel/__init__.py", line 5, in <module>
    from .connect import *  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shashikumarezhil/anaconda3/lib/python3.11/site-packages/ipykernel/connect.py", line 11, in <module>
    import jupyter_client
  File "/Users/shashikumarezhil/anaconda3/lib/python3.11/site-packages/jupyter_client/__init__.py", line 8, in <module>
    from .asynchronous import AsyncKernelClient  # noqa
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/shashikumarezhil/anaconda3/lib/python3.11/site-packages/jupyter_client/asynchronous/__init__.py", line 1, in <module>
    from .client import AsyncKernel

SystemExit: 1

/Users/shashikumarezhil/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
import folium
from math import radians, sin, cos, sqrt, atan2

# Function to calculate the distance between two latitude and longitude points using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in kilometers
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c  # Return distance in kilometers

# Function to find the nearest station from a clicked point
def find_nearest_station(click_lat, click_lon, records):
    min_dist = float('inf')  # Set to infinity at the start
    nearest_station = None

    for record in records:
        station_coords = (record['latitude'], record['longitude'])
        dist = haversine(click_lat, click_lon, *station_coords)
        if dist < min_dist:
            min_dist = dist
            nearest_station = record

    return nearest_station, min_dist

# Sample records with station name, latitude, and longitude
records = [
    {'Station Name': 'Station 1', 'latitude': 41.5025, 'longitude': -72.69997},
    {'Station Name': 'Station 2', 'latitude': 41.7025, 'longitude': -72.89997},
    {'Station Name': 'Station 3', 'latitude': 41.6025, 'longitude': -72.79997},
    {'Station Name': 'Station 4', 'latitude': 41.5028, 'longitude': -72.69987},
    {'Station Name': 'Station 5', 'latitude': 41.5030, 'longitude': -72.69977},
]

# Initialize map centered around the first station
map = folium.Map(location=[41.5025, -72.69997], zoom_start=9)

# Add markers for the stations
for record in records:
    folium.Marker(
        location=[record['latitude'], record['longitude']],
        popup=record['Station Name']
    ).add_to(map)

# Add a div to display nearest station and distance
nearest_station_html = """
<div id="nearest-station" style="position: fixed; 
                                 bottom: 50px; 
                                 left: 50px; 
                                 width: 300px; 
                                 padding: 10px; 
                                 background-color: white; 
                                 border: 2px solid black; 
                                 z-index: 1000;">
    Click on the map to find the nearest station.
</div>
"""

map.get_root().html.add_child(folium.Element(nearest_station_html))

# Add a LatLngPopup to capture clicks on the map
map.add_child(folium.LatLngPopup())

# Add JavaScript to process the click event and find the nearest station
click_js = """
function(e) {
    var lat = e.latlng.lat;
    var lon = e.latlng.lng;

    // Calculate the nearest station on the server
    fetch('/find-nearest?lat=' + lat + '&lon=' + lon)
        .then(response => response.json())
        .then(data => {
            // Update the nearest station info
            document.getElementById("nearest-station").innerHTML = 
                "Nearest Station: " + data.station_name + "<br>" +
                "Distance: " + data.distance.toFixed(2) + " km";
        });
}
"""

# Add the JS to trigger on map click
map.get_root().script.add_child(folium.Element(f"""
<script>
    var map = document.getElementById('map');
    map.on('click', {click_js});
</script>
"""))

# Render the map
map
